In [2]:
# necessary imports

import re
import csv
import pandas as pd
import numpy as np

In [25]:
# loading data into a data frame

filename = r"C:\Users\manas\Documents\Master Thesis\code\Manasa2022\skill_id\skill_id_data_1million.csv"
data_df = pd.read_csv(filename,sep=',')

display(data_df)

,skill_id
0,"1589682.0,972614.0,959423.0,958546.0,7547697.0..."
1,"954981.0,954651.0,972914.0,955590.0,958410.0,1..."
2,"955590.0,972787.0,957701.0,1906300.0,972842.0,..."
3,"958250.0,955622.0,1587760.0,958678.0,972565.0"
4,"1588624.0,972568.0,974186.0,972961.0,972823.0,..."
...,...
568265,972795.0
568266,959351.0
568267,"972830.0,977782.0,1587495.0,972787.0,957701.0,..."
568268,"972533.0,956539.0,977782.0,1587495.0,958531.0,..."


In [4]:
from collections import Counter
import itertools
result = Counter(",".join(data['skill_id'].values.tolist()).split(",")).items()
#result

In [8]:
#considering the values whose count is greater than 1
df_data = pd.DataFrame(result, columns=['skill_id', 'countValue'])
final_df = df_data.loc[df_data['countValue'] > 1]
final_df

,skill_id,countValue
0,1589682.0,597
1,972614.0,5659
2,959423.0,4719
3,958546.0,16223
4,7547697.0,757
...,...,...
5116,956342.0,9
5142,959066.0,4
5184,958514.0,4
5238,955832.0,4


In [26]:
# we need to convert each sentence into a more desirable format
    
# define the desired splitting condition
split_when = ","
remove_when = "[^,.\w]"

def convert_sentence(x):
    x = re.sub(remove_when, '', x).lower()
    return re.split(split_when, x)

data_df['skill_id'] = data_df['skill_id'].apply(convert_sentence)
display(data_df)

,skill_id
0,"[1589682.0, 972614.0, 959423.0, 958546.0, 7547..."
1,"[954981.0, 954651.0, 972914.0, 955590.0, 95841..."
2,"[955590.0, 972787.0, 957701.0, 1906300.0, 9728..."
3,"[958250.0, 955622.0, 1587760.0, 958678.0, 9725..."
4,"[1588624.0, 972568.0, 974186.0, 972961.0, 9728..."
...,...
568265,[972795.0]
568266,[959351.0]
568267,"[972830.0, 977782.0, 1587495.0, 972787.0, 9577..."
568268,"[972533.0, 956539.0, 977782.0, 1587495.0, 9585..."


In [7]:
# count how many unique words are there to create the co-occurrence matrix
# right now the number of words is set to 500
# the actual number is
# n_words = 6338804
# n_unique = 118194
# remove the limit just delete "[:500]" in the below line
# but it's not recommended because the runtime would be around 300 years
# to go around this I recommended somethings you can look into in the last cell

words = list(col for row in data['skill_id'] for col in row)
n_words = len(words)

unique_words = list(set(words))
n_unique = len(unique_words)

(n_words, n_unique)

(5480418, 5334)

In [21]:
import time

# get the start time
st = time.time()

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1,1),lowercase= False)
X_counts = count_vect.fit_transform(manasa['skill_id'])
#X_counts
Xc = (X_counts.T * X_counts)
#Xc
Xc.setdiag(0)

# get the end time
et = time.time()

# get the execution time
# get the execution time
res = et - st
final_res = res / 60
print('Execution time:', final_res, 'minutes')

Execution time: 0.27448784510294594 minutes


In [27]:
final_res

0.27448784510294594

In [18]:
sk_df = pd.DataFrame(Xc.todense(), columns=count_vect.get_feature_names(), index=count_vect.get_feature_names())
sk_df

,10124166,10209981,10229057,10242030,10309339,10391083,10430108,10521690,10531188,10540923,...,977801,977802,977803,977804,977805,977806,977807,977882,9802196,9856113
10124166,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10209981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10229057,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
10242030,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10309339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977806,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
977807,0,0,0,0,0,49,0,0,0,0,...,36,101,0,0,0,0,0,7,0,0
977882,0,0,0,0,0,0,0,2,0,0,...,14,2,0,0,0,0,7,0,0,0
9802196,0,0,0,0,0,0,5,0,1,0,...,1,0,0,1,0,0,0,0,0,6


In [11]:
import itertools

# get all possible pairs of (skillid1, skillid2)
edges = list(itertools.combinations(sk_df.columns, 2))  

# find associated weights in the original df
edges_with_weights = [(node1, node2, sk_df.loc[node1][node2]) for (node1, node2) in edges]

# put it all in a new dataframe
new_df = pd.DataFrame(edges_with_weights, columns=["source", "target", "weight"])

In [16]:
new_df.tail(100)

,source,target,weight
5129527,20850135,7549596,0
5129412,20850135,7547921,0
5129438,20850135,7548204,0
5129441,20850135,7548263,0
5129442,20850135,7548285,0
...,...,...,...
5129486,20850135,7548831,0
5129487,20850135,7548834,0
5129488,20850135,7548835,0
5129489,20850135,7548865,0


In [15]:
new_df.sort_values(by=['weight'], inplace=True, ascending=False)
new_df.head()

,source,target,weight
13406865,958531,977782,43850
12791490,957701,972468,41297
13920348,972468,972787,39423
12791667,957701,972787,35871
13930105,972533,977782,33373


In [18]:
new_df.to_csv(r"C:\Users\manas\Documents\Master Thesis\code\Manasa2022\skill_id\co-occurence\co-occurence.csv",header=True,index=False)

In [40]:
final_df.rename(columns={'skillid1': 'source', 'skillid2': 'target'}, inplace=True)
final_df

C:\Users\manas\anaconda3\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,source,target,count
117,10124166,1587408,1
118,10124166,1587409,1
121,10124166,1587412,1
140,10124166,1587431,1
159,10124166,1587453,1
...,...,...,...
14217746,977802,977807,101
14217747,977802,977882,2
14217761,977804,9802196,1
14217772,977807,977882,7


In [22]:
# loading unique skill_id data into a data frame

filename = r"C:\Users\manas\Documents\Master Thesis\code\Manasa2022\skill_id\testing\skill_id_from_word2vec.csv"
data = pd.read_csv(filename)

display(data)

,skill_id
0,7.0
1,14.0
2,21.0
3,22.0
4,34.0
...,...
4870,67658437.0
4871,67673382.0
4872,67673383.0
4873,67673384.0
